In [1]:
from google_maps_geocoder.geocoder import GoogleGeocoder
import pandas as pd
import os

# Initialize the GoogleGeocoder
api_key = os.getenv("GOOGLE_API_KEY")
geocoder = GoogleGeocoder(api_key)

# Load the dataset
input_data = pd.read_csv(r".\example_address.csv")

# Clean and prepare the data
destinations, needs_geocoding = geocoder.cleanup_pd(input_data)

# Perform geocoding
if needs_geocoding:
    final_dest_df = geocoder.geocode_addresses(destinations, needs_geocoding)
    final_dest_df.to_csv('geocoded_results.csv', index=False)
else:
    print("Data already contains coordinates.")

tef = geocoder.geocode_single('1099 Stewart Street,Seattle,WA')
# df = pd.json_normalize(tef)
tef


Successfully retrieved results for address: 123 Maple St,Boston,MA,2110
Successfully retrieved results for address: 456 Oak Ave,New York,NY,10001
Successfully retrieved results for address: 789 Pine Rd,Chicago,IL,60601
Successfully retrieved results for address: 101 Birch Blvd,San Francisco,CA,94105
Successfully retrieved results for address: 202 Cedar Ln,Los Angeles,CA,90001
Successfully retrieved results for address: 303 Elm St,Seattle,WA,98101
Successfully retrieved results for address: 404 Walnut Dr,Denver,CO,80202
Successfully retrieved results for address: 505 Redwood Way,Austin,TX,73301
Successfully retrieved results for address: 606 Ash St,Miami,FL,33101
Successfully retrieved results for address: 707 Palm Ave,San Diego,CA,92101
Successfully retrieved results for address: 1099 Stewart Street,Seattle,WA


,formatted_address,latitude,longitude,accuracy,google_place_id,type,postcode,input_string,number_of_results,status,response
0,"1099 Stewart St, Seattle, WA 98101, USA",47.616661,-122.332716,ROOFTOP,ChIJgdY5t4oVkFQRMfVIB7Js_4s,street_address,98101,"1099 Stewart Street,Seattle,WA",1,OK,None
